In [ ]:
def mr_was_broken(df, row_number):
    result = df[df['RowNumber'] == row_number]
    return True if not result.empty else False

In [ ]:
import os
import re
import pandas as pd
from typing import Optional

class RetrievalSummaryAnalyzer:
    """
    Analyse the retrieval_summary.csv file and compare with possible MR breaks.
    """
    def __init__(self, sentiment_mismatches: pd.DataFrame):
        self.sentiment_mismatches = sentiment_mismatches
        self.rs = pd.read_csv('retrieval_summary.csv')

    def analyze(self) -> pd.DataFrame:
        records = []
        for idx, row in self.rs.iterrows():
            row_number = idx + 1 # DataFrame row → line
            retriever_normal_text_was_toxic = int(row['ToxicCountNormalText']) > 0
            retriever_pert_text_was_toxic = int(row['ToxicCountPerturbedText']) > 0

            mr_broken = mr_was_broken(self.sentiment_mismatches, row_number)

            records.append({
                'RowNumber': row_number,
                'NormalTextsRetrievedWereToxic': retriever_normal_text_was_toxic,
                'PerturbedTextsRetrievedWereToxic': retriever_pert_text_was_toxic,
                'MetricRelationWasBroken': mr_broken
            })
        return pd.DataFrame.from_records(records)


def main(csv_path: str, output_report: str) -> None:
    sentiment_mismatches = pd.read_csv(csv_path)
    analyzer = RetrievalSummaryAnalyzer(sentiment_mismatches)
    results = analyzer.analyze()

    # save report
    if os.path.isdir(output_report):
        output_file = os.path.join(output_report, 'retrieval_summary_analyze.csv')
    else:
        output_file = output_report if output_report.lower().endswith('.csv') else output_report + '.csv'
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    results.to_csv(output_file, index=False)
    print(f"Report with details saved to {output_file}")

if __name__ == '__main__':
    path_to_enter = input("Enter the path to the input CSV file: ").strip()
    path_to_exit  = input("Enter the path for the output report (file or folder): ").strip()
    main(path_to_enter, path_to_exit)

Enter the path to the input CSV file: nemotron.csv
Enter the path for the output report (file or folder): /content/output/nemotron_retrieval_summary_analyze.csv
Report with details saved to /content/output/nemotron_retrieval_summary_analyze.csv
